# Imports y definiciones

In [1]:
import os, re, json
import numpy as np
import pandas as pd
import cvxpy as cp
from itertools import combinations
from ast import literal_eval
from collections import defaultdict
from scipy.spatial import distance

In [2]:
databases = 'C:/Users/Chris-Brota/Desktop/Databases'
display(os.listdir(databases))

# Se elige el JSON
db = f'{databases}/{os.listdir(databases)[1]}'
db

['CIQUAL',
 'DUKE',
 'FNDDS',
 'foodb_2020_04_07_csv',
 'foodb_2020_04_07_json',
 'foodb_server_dump_2020_4_21.sql']

'C:/Users/Chris-Brota/Desktop/Databases/DUKE'

# Tablas a sacar

Estas tablas en general no poseen mucha de la información relevante que se necesita, por lo cual no se considerará en el MVP0. 

In [3]:
dont_consider = (109, 168, 219, 905, 947)

# Características de interés

In [4]:
compounds_mvp0 = ['Protein', 'Calcium, Ca', 'Iron, Fe', 'Phosphorus, P', 
                  'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu', 
                  'Thiamin', 'Riboflavin', 'Niacin', 'Vitamin B6', 'Folic acid']

# Creación de las recetas

In [5]:
def get_recipe_portions(ingredients, solver=cp.ECOS, verbose=False):
    def _objective_table():
        # Definición de la tabla nutricional objetivo
        obj_table = list()
        
        with open('summary/objective_table.csv', 'r', encoding='utf8') as file:
            # Saltar el encabezado
            file.readline()

            for line in file:
                compound_to_rev = line.strip().split(';')[0].replace('\"', '')

                if compound_to_rev in compounds_mvp0:
                    obj_table.append((compound_to_rev, float(line.strip().split(';')[1])))
                    
        return obj_table
    
    
    def _recipes_table():
        # Creación del problema de optimización para la definición de la receta
        recipe_matrix = list()

        for ing_i in ingredients:
            # Definición de la información del ingrediente i
            ing_i_list = list()

            with open(f'summary/foodTables/{ing_i}', 'r', encoding='utf8') as file:
                for line in file:
                    comp_i, value_i = line.strip().split(';')
                    ing_i_list.append((comp_i, float(value_i)))

            # Agregando a la matriz de recetas
            recipe_matrix.append(ing_i_list)
    
        return recipe_matrix
    
    
    def _model():
        # Variable de cantidad
        x = cp.Variable(len(ingredients))
        
        # Parámetros
        T_obj = cp.Parameter(shape=obj_table_arr.shape)
        T_obj.value = obj_table_arr
        T_rec = cp.Parameter(shape=recipe_matrix_arr.shape)
        T_rec.value = recipe_matrix_arr

        # Restricciones
        constraints = [x >= 0]

        # Definición de la función objetivo
        objective_func = cp.Minimize(cp.sum_squares((T_obj - T_rec @ x)))

        # Definición del problema de optimización
        prob = cp.Problem(objective=objective_func, constraints=constraints)
        
        # Calculando el óptimo
        result = prob.solve(solver=solver, verbose=verbose)
        
        return result, x.value
     
    
    def _cosine_similarity(a, b):
        return 1 - distance.cosine(a, b)
    
    
    def _euclidean_distance(a, b):
        return np.sqrt(np.sum((a - b) ** 2))
    
    
    # Obteniendo las tablas
    obj_table = _objective_table()
    recipe_matrix = _recipes_table()
    
    # Transformando a arrays
    obj_table_arr = np.array(obj_table)[:,1].astype(float)
    recipe_matrix_arr = np.array(recipe_matrix)[:,:,1].astype(float).T
    
    # Obtención de los valores del modelo
    result, x = _model()
    
    # Obteniendo la distancia euclidiana y coseno
    euclidean_dist = _euclidean_distance(obj_table_arr, recipe_matrix_arr @ x)
    cosine_sim     = _cosine_similarity(obj_table_arr, recipe_matrix_arr @ x)
    
    return result, x, (euclidean_dist, cosine_sim)

In [6]:
# Número de ingredientes máximos por receta
compounds_max = 5

# Definición de los ingredientes
ingredients = os.listdir('summary/foodTables')

# Definición de las diferentes combinaciones de recetas en base a los ingredientes
recipes = combinations(ingredients, compounds_max)

# Rutina de ponderación de recetas

In [7]:
# Definición de la información de las recetas
recipes_results = list()

# Para cada receta en recetas
for recipe_i in recipes:
    # Obtención de las porciones de las recetas
    result, x, metrics = get_recipe_portions(recipe_i, solver=cp.CVXOPT, 
                                             verbose=False)
    
    
    recipes_results.append((recipe_i, result, x, metrics))

# Ordenando por similaridad de recetas

In [8]:
recipes_ordered = [i for i in recipes_results]
recipes_ordered.sort(key=lambda x: x[3][1], reverse=True)

In [9]:
for num, i in enumerate(recipes_ordered):
    print(i)
    # Formateando el texto
    text = ','.join([t.split('_')[1][:-4] for t in i[0]])
    
    # Desplegando los resultados
    print(f'{num+1};{text};{i[3][1]:.10f}')

(('165_Common sage.txt', '341_Coconut.txt', '562_Shiitake.txt', '666_Salt.txt', '748_Hibiscus tea.txt'), 6.532046639823978e-09, array([2.89345599e-04, 1.49806470e-03, 6.87047583e-05, 1.25395511e-03,
       5.77965445e-01]), (8.082107794272468e-05, 0.9995170204842697))
1;Common sage,Coconut,Shiitake,Salt,Hibiscus tea;0.9995170205
(('183_Common thyme.txt', '341_Coconut.txt', '372_Ginkgo nuts.txt', '666_Salt.txt', '748_Hibiscus tea.txt'), 6.5804168849935455e-09, array([7.03920086e-04, 2.21067622e-03, 2.99639929e-05, 1.24925066e-03,
       5.69893200e-01]), (8.111976876812177e-05, 0.9995136550376988))
2;Common thyme,Coconut,Ginkgo nuts,Salt,Hibiscus tea;0.9995136550
(('165_Common sage.txt', '206_Ginger.txt', '586_Cinnamon.txt', '666_Salt.txt', '748_Hibiscus tea.txt'), 6.6144442090805975e-09, array([3.90559013e-04, 1.25795737e-04, 3.30274168e-03, 1.28120497e-03,
       5.60531354e-01]), (8.132923342243303e-05, 0.9995124666008693))
3;Common sage,Ginger,Cinnamon,Salt,Hibiscus tea;0.9995124666

(('384_Horseradish tree.txt', '562_Shiitake.txt', '569_Maitake.txt', '58_Coffee.txt', '666_Salt.txt'), 3.438208728308434e-08, array([0.00607727, 0.01150707, 0.06381351, 0.0034136 , 0.00132036]), (0.00018542407417345878, 0.9974558684871518))
882;Horseradish tree,Shiitake,Maitake,Coffee,Salt;0.9974558685
(('206_Ginger.txt', '372_Ginkgo nuts.txt', '666_Salt.txt', '68_Turmeric.txt', '748_Hibiscus tea.txt'), 3.479118524647979e-08, array([0.01095704, 0.00772638, 0.001396  , 0.00186153, 0.33572757]), (0.00018652395354613248, 0.9974547440329475))
883;Ginger,Ginkgo nuts,Salt,Turmeric,Hibiscus tea;0.9974547440
(('206_Ginger.txt', '384_Horseradish tree.txt', '562_Shiitake.txt', '569_Maitake.txt', '666_Salt.txt'), 3.442914293340767e-08, array([0.00796429, 0.00569958, 0.01201059, 0.0579576 , 0.00129996]), (0.0001855509173607279, 0.9974523851333648))
884;Ginger,Horseradish tree,Shiitake,Maitake,Salt;0.9974523851
(('183_Common thyme.txt', '569_Maitake.txt', '586_Cinnamon.txt', '666_Salt.txt', '748_Hi

(('341_Coconut.txt', '40_Pepper.txt', '562_Shiitake.txt', '666_Salt.txt', '748_Hibiscus tea.txt'), 4.89593042415388e-08, array([0.02183728, 0.01150418, 0.01548308, 0.00107233, 0.23313351]), (0.0002212674947694279, 0.99679855764974))
980;Coconut,Pepper,Shiitake,Salt,Hibiscus tea;0.9967985576
(('246_Triticale.txt', '562_Shiitake.txt', '569_Maitake.txt', '58_Coffee.txt', '666_Salt.txt'), 4.537462285726845e-08, array([0.00466623, 0.01686488, 0.05476905, 0.00582453, 0.00138752]), (0.00021301319878652697, 0.996797723502631))
981;Triticale,Shiitake,Maitake,Coffee,Salt;0.9967977235
(('165_Common sage.txt', '206_Ginger.txt', '569_Maitake.txt', '586_Cinnamon.txt', '666_Salt.txt'), 4.318364367707223e-08, array([0.0015961 , 0.00782467, 0.0843569 , 0.01392316, 0.00134949]), (0.00020780674598547621, 0.9967946100168908))
982;Common sage,Ginger,Maitake,Cinnamon,Salt;0.9967946100
(('372_Ginkgo nuts.txt', '384_Horseradish tree.txt', '40_Pepper.txt', '569_Maitake.txt', '666_Salt.txt'), 4.410288859048715e

(('183_Common thyme.txt', '246_Triticale.txt', '384_Horseradish tree.txt', '40_Pepper.txt', '68_Turmeric.txt'), 1.8131861898152405e-07, array([0.00365796, 0.00529954, 0.00475812, 0.02439452, 0.00158636]), (0.0004258152404289025, 0.9864782995077396))
1827;Common thyme,Triticale,Horseradish tree,Pepper,Turmeric;0.9864782995
(('183_Common thyme.txt', '40_Pepper.txt', '562_Shiitake.txt', '586_Cinnamon.txt', '748_Hibiscus tea.txt'), 1.814888014680146e-07, array([0.00245756, 0.0289853 , 0.01030524, 0.00825574, 0.18160923]), (0.00042601502493223707, 0.9864782517311853))
1828;Common thyme,Pepper,Shiitake,Cinnamon,Hibiscus tea;0.9864782517
(('372_Ginkgo nuts.txt', '384_Horseradish tree.txt', '40_Pepper.txt', '562_Shiitake.txt', '58_Coffee.txt'), 1.815342759922697e-07, array([0.00254084, 0.00997909, 0.0295475 , 0.00378516, 0.00180288]), (0.00042606839356172584, 0.9864689252325459))
1829;Ginkgo nuts,Horseradish tree,Pepper,Shiitake,Coffee;0.9864689252
(('246_Triticale.txt', '372_Ginkgo nuts.txt',

1881;Rosemary,Coconut,Pepper,Cinnamon,Hibiscus tea;0.9862486519
(('246_Triticale.txt', '40_Pepper.txt', '586_Cinnamon.txt', '58_Coffee.txt', '68_Turmeric.txt'), 1.843421315140221e-07, array([0.00886725, 0.02760972, 0.0101214 , 0.00219017, 0.00062768]), (0.000429350825682241, 0.9862466819386649))
1882;Triticale,Pepper,Cinnamon,Coffee,Turmeric;0.9862466819
(('246_Triticale.txt', '341_Coconut.txt', '384_Horseradish tree.txt', '40_Pepper.txt', '586_Cinnamon.txt'), 1.9241567892062004e-07, array([0.00395627, 0.01453367, 0.00352793, 0.02600698, 0.010834  ]), (0.00043865211605624343, 0.9862431459492375))
1883;Triticale,Coconut,Horseradish tree,Pepper,Cinnamon;0.9862431459
(('372_Ginkgo nuts.txt', '384_Horseradish tree.txt', '40_Pepper.txt', '586_Cinnamon.txt', '58_Coffee.txt'), 1.8460303647925323e-07, array([0.00158671, 0.01057216, 0.02739568, 0.0067488 , 0.00195222]), (0.00042965455482195603, 0.9862390477599963))
1884;Ginkgo nuts,Horseradish tree,Pepper,Cinnamon,Coffee;0.9862390478
(('159_Ros

2631;Triticale,Ginkgo nuts,Horseradish tree,Cinnamon,Coffee;0.9846687915
(('246_Triticale.txt', '341_Coconut.txt', '384_Horseradish tree.txt', '586_Cinnamon.txt', '68_Turmeric.txt'), 2.0551554139017056e-07, array([0.00943038, 0.01317781, 0.00585293, 0.00756119, 0.0009034 ]), (0.0004533382196441974, 0.9846687586237082))
2632;Triticale,Coconut,Horseradish tree,Cinnamon,Turmeric;0.9846687586
(('372_Ginkgo nuts.txt', '569_Maitake.txt', '58_Coffee.txt', '68_Turmeric.txt', '748_Hibiscus tea.txt'), 2.0544355610363498e-07, array([0.00452153, 0.01391335, 0.00489117, 0.00104112, 0.38088204]), (0.00045325881800979337, 0.9846685531765091))
2633;Ginkgo nuts,Maitake,Coffee,Turmeric,Hibiscus tea;0.9846685532
(('159_Rosemary.txt', '341_Coconut.txt', '372_Ginkgo nuts.txt', '562_Shiitake.txt', '748_Hibiscus tea.txt'), 2.0560857459666217e-07, array([0.00552918, 0.0154051 , 0.00532729, 0.00925107, 0.29653991]), (0.00045344081708273925, 0.9846671226282995))
2634;Rosemary,Coconut,Ginkgo nuts,Shiitake,Hibisc

(('165_Common sage.txt', '206_Ginger.txt', '246_Triticale.txt', '569_Maitake.txt', '586_Cinnamon.txt'), 2.0640227178132312e-07, array([0.0008823 , 0.00189805, 0.01597658, 0.00838132, 0.00395426]), (0.0004543151678970482, 0.9845848804589746))
2727;Common sage,Ginger,Triticale,Maitake,Cinnamon;0.9845848805
(('183_Common thyme.txt', '206_Ginger.txt', '341_Coconut.txt', '384_Horseradish tree.txt', '68_Turmeric.txt'), 2.0648646179363516e-07, array([0.00187365, 0.00408209, 0.01079815, 0.01613649, 0.00070572]), (0.0004544078144064373, 0.9845838236322358))
2728;Common thyme,Ginger,Coconut,Horseradish tree,Turmeric;0.9845838236
(('159_Rosemary.txt', '165_Common sage.txt', '341_Coconut.txt', '569_Maitake.txt', '748_Hibiscus tea.txt'), 2.065540528864183e-07, array([0.00682829, 0.00103918, 0.01791167, 0.01804542, 0.31284161]), (0.0004544821810438978, 0.9845820734868891))
2729;Rosemary,Common sage,Coconut,Maitake,Hibiscus tea;0.9845820735
(('159_Rosemary.txt', '183_Common thyme.txt', '372_Ginkgo nu

(('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '569_Maitake.txt', '748_Hibiscus tea.txt'), 2.3035596463401735e-07, array([0.00683505, 0.00131391, 0.00401162, 0.03289147, 0.23050204]), (0.00047995412763514946, 0.9836261780428663))
3587;Rosemary,Common sage,Common thyme,Maitake,Hibiscus tea;0.9836261780
(('159_Rosemary.txt', '165_Common sage.txt', '183_Common thyme.txt', '372_Ginkgo nuts.txt', '569_Maitake.txt'), 2.1935187182287483e-07, array([0.00233097, 0.00065389, 0.00166851, 0.00857281, 0.08671807]), (0.00046835015941373907, 0.9836233111427757))
3588;Rosemary,Common sage,Common thyme,Ginkgo nuts,Maitake;0.9836233111
(('159_Rosemary.txt', '206_Ginger.txt', '384_Horseradish tree.txt', '58_Coffee.txt', '748_Hibiscus tea.txt'), 2.1941735688967278e-07, array([0.00606196, 0.00708767, 0.00306539, 0.00487935, 0.33149677]), (0.0004684200645677689, 0.9836229851456132))
3589;Rosemary,Ginger,Horseradish tree,Coffee,Hibiscus tea;0.9836229851
(('159_Rosemary.txt', '372_Ginkgo

3630;Common sage,Triticale,Ginkgo nuts,Maitake,Cinnamon;0.9835567086
(('246_Triticale.txt', '372_Ginkgo nuts.txt', '562_Shiitake.txt', '569_Maitake.txt', '68_Turmeric.txt'), 2.2070382274819972e-07, array([0.00332785, 0.00681371, 0.0161562 , 0.05064727, 0.00101417]), (0.0004697912544398839, 0.9835534910167869))
3631;Triticale,Ginkgo nuts,Shiitake,Maitake,Turmeric;0.9835534910
(('206_Ginger.txt', '384_Horseradish tree.txt', '569_Maitake.txt', '586_Cinnamon.txt', '58_Coffee.txt'), 2.201580356453821e-07, array([0.00478749, 0.01176968, 0.02919576, 0.00853272, 0.00338829]), (0.0004692100123030007, 0.9835526976953511))
3632;Ginger,Horseradish tree,Maitake,Cinnamon,Coffee;0.9835526977
(('165_Common sage.txt', '183_Common thyme.txt', '341_Coconut.txt', '40_Pepper.txt', '586_Cinnamon.txt'), 2.2355228026620782e-07, array([0.00104726, 0.00416153, 0.02038223, 0.03627197, 0.01041561]), (0.00047281315576685025, 0.9835522196226191))
3633;Common sage,Common thyme,Coconut,Pepper,Cinnamon;0.9835522196
((

In [12]:
for num, i in enumerate(recipes_ordered):
    # Desplegando los resultados en bruto
    display(i)

(('165_Common sage.txt',
  '341_Coconut.txt',
  '562_Shiitake.txt',
  '666_Salt.txt',
  '748_Hibiscus tea.txt'),
 6.532046639823978e-09,
 array([2.89345599e-04, 1.49806470e-03, 6.87047583e-05, 1.25395511e-03,
        5.77965445e-01]),
 (8.082107794272468e-05, 0.9995170204842697))

(('183_Common thyme.txt',
  '341_Coconut.txt',
  '372_Ginkgo nuts.txt',
  '666_Salt.txt',
  '748_Hibiscus tea.txt'),
 6.5804168849935455e-09,
 array([7.03920086e-04, 2.21067622e-03, 2.99639929e-05, 1.24925066e-03,
        5.69893200e-01]),
 (8.111976876812177e-05, 0.9995136550376988))

(('165_Common sage.txt',
  '206_Ginger.txt',
  '586_Cinnamon.txt',
  '666_Salt.txt',
  '748_Hibiscus tea.txt'),
 6.6144442090805975e-09,
 array([3.90559013e-04, 1.25795737e-04, 3.30274168e-03, 1.28120497e-03,
        5.60531354e-01]),
 (8.132923342243303e-05, 0.9995124666008693))

(('183_Common thyme.txt',
  '372_Ginkgo nuts.txt',
  '562_Shiitake.txt',
  '666_Salt.txt',
  '748_Hibiscus tea.txt'),
 6.647413447051341e-09,
 array([9.28769405e-04, 4.08682963e-05, 1.15595236e-03, 1.24064057e-03,
        5.68577437e-01]),
 (8.15316714354081e-05, 0.9995117976671176))

(('165_Common sage.txt',
  '40_Pepper.txt',
  '586_Cinnamon.txt',
  '666_Salt.txt',
  '748_Hibiscus tea.txt'),
 6.722543070546089e-09,
 array([4.27589311e-04, 3.24028898e-04, 3.13264068e-03, 1.27357296e-03,
        5.59069175e-01]),
 (8.199111580254344e-05, 0.9995042034560255))

...

(('165_Common sage.txt',
  '206_Ginger.txt',
  '246_Triticale.txt',
  '372_Ginkgo nuts.txt',
  '68_Turmeric.txt'),
 2.2692684859432174e-07,
 array([0.00277008, 0.00724481, 0.00882553, 0.00838893, 0.00201074]),
 (0.0004763683958810888, 0.9830885621985829))

(('159_Rosemary.txt',
  '183_Common thyme.txt',
  '206_Ginger.txt',
  '384_Horseradish tree.txt',
  '569_Maitake.txt'),
 2.2631520947799405e-07,
 array([0.00600216, 0.00358123, 0.0068048 , 0.00963434, 0.02996177]),
 (0.00047572598150405243, 0.9830855045937577))

(('246_Triticale.txt',
  '372_Ginkgo nuts.txt',
  '384_Horseradish tree.txt',
  '569_Maitake.txt',
  '58_Coffee.txt'),
 2.2638174019008847e-07,
 array([0.00283278, 0.00777929, 0.00328791, 0.04950027, 0.00666588]),
 (0.00047579590182145166, 0.9830842418371681))

(('159_Rosemary.txt',
  '165_Common sage.txt',
  '206_Ginger.txt',
  '569_Maitake.txt',
  '68_Turmeric.txt'),
 2.2663291801015138e-07,
 array([0.00309171, 0.001021  , 0.00311362, 0.10790509, 0.00068478]),
 (0.00047605978407144555, 0.9830834990897779))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Prints presentación

In [25]:
for name in os.listdir('summary/foodTables')[:4]:
    print(name)
    with open(f'summary/foodTables/{name}', 'r', encoding='utf8') as file:
        for line in file:
            value = float(line.strip().split(';')[1])
            print(f'{line.strip().split(";")[0]};{value:.10f}')
            
    print()

159_Rosemary.txt
Protein;0.0445113333
Calcium, Ca;0.0081686250
Iron, Fe;0.0001838750
Phosphorus, P;0.0006912500
Potassium, K;0.0082432500
Sodium, Na;0.0003867500
Zinc, Zn;0.0000212250
Copper, Cu;0.0000042550
Thiamin;0.0000019533
Riboflavin;0.0000029000
Niacin;0.0000096850
Vitamin B6;0.0000080400
Folic acid;0.0002735400

165_Common sage.txt
Protein;0.1337666667
Calcium, Ca;0.0154492500
Iron, Fe;0.0002543500
Phosphorus, P;0.0010025000
Potassium, K;0.0155216667
Sodium, Na;0.0004005000
Zinc, Zn;0.0000530000
Copper, Cu;0.0000075350
Thiamin;0.0000075400
Riboflavin;0.0000034300
Niacin;0.0000583500
Vitamin B6;0.0000269000
Folic acid;0.0000020550

183_Common thyme.txt
Protein;0.0819000000
Calcium, Ca;0.0137224167
Iron, Fe;0.0008225000
Phosphorus, P;0.0015320833
Potassium, K;0.0077707500
Sodium, Na;0.0006092500
Zinc, Zn;0.0000398833
Copper, Cu;0.0000073600
Thiamin;0.0000028050
Riboflavin;0.0000124000
Niacin;0.0000378560
Vitamin B6;0.0000044900
Folic acid;0.0005987231

206_Ginger.txt
Protein;0.03

In [28]:
# Definición de la tabla nutricional objetivo
obj_table = list()

with open('summary/objective_table.csv', 'r', encoding='utf8') as file:
    # Saltar el encabezado
    file.readline()

    for line in file:
        compound_to_rev = line.strip().split(';')[0].replace('\"', '')

        if compound_to_rev in compounds_mvp0:
            value = float(line.strip().split(";")[1])
            print(f'{compound_to_rev};{value:.10f}')

Protein;0.0025576037
Calcium, Ca;0.0000535714
Iron, Fe;0.0000002160
Phosphorus, P;0.0000128571
Potassium, K;0.0000378571
Sodium, Na;0.0004450000
Zinc, Zn;0.0000009505
Copper, Cu;0.0000000297
Thiamin;0.0000001760
Riboflavin;0.0000031852
Niacin;0.0000460602
Vitamin B6;0.0000099784
Folic acid;0.0000002386
